# OpenAI: Text to Speech

In [1]:
import os
from dotenv import load_dotenv

# 토큰 정보로드
api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()

True

In [ ]:
from openai import OpenAI

client = OpenAI()

## 1. Text To Speech(TTS)

### Client 생성

- `client` 는 OpenAI 모듈로 생성된 인스턴스 입니다.

[주의] 아래의 코드에서 오류가 난다면 API 키의 오류일 가능성이 높습니다.


In [4]:
speech_file_path = "tts_audio.mp3"

response = client.audio.speech.create(
    model="tts-1",
    input="아~ 오늘 파이썬 배우기 정말 좋은 날이네~",
    voice="shimmer",
    response_format="mp3",
    speed=2.0,
)

response.stream_to_file(speech_file_path)

/tmp/ipykernel_4084825/1531316983.py:11: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  response.stream_to_file(speech_file_path)


In [5]:
from IPython.display import Audio

Audio(speech_file_path)

## 2. Speech To Text(STT)

In [6]:
audio_file = open("data/채용면접_샘플_01.wav", "rb")
transcript = client.audio.transcriptions.create(
    file=audio_file,
    model="whisper-1",
    language="ko",
    response_format="text",
    temperature=0.0,
)

In [7]:
# 결과물 출력
print(transcript)

지금 생각해보면 가장 기억에 남는 것이 외환위기 때의 경험입니다. 외환위기 때 나라뿐 아니라 회사에서는 달러가 많이 부족했고 우리는 수출을 하기 위해서 원자재라든지 대금지급 혹은 또 우리가 수출한 물건에 대한 대금을 받아야 되는 상황이었습니다. 일단은 우리가 해외로부터 받아야 될 때는 최대한 그것을 달러로 받았고 반대로 우리가 지불해야 될 것은 가능한 한 원자재를 통해서 지급을 했습니다. 그 원자재라는 것이 결국은 반도체 쪽이었는데 우리나라에서 가장 나름 손쉽게 구하면서도 꼭 필요한 제품인 반도체를 원자재값 대응으로 물건을 주면서 반대로 해외에서 우리가 받아야 될 것은 달러로 받으면서 그 환율차를 가장 줄일 수가 있었습니다. 그것으로 인해서 약 6개월 동안에 나름 회사에서는 달러 지출을 막을 수 있었고 그때 그나마 달러를 회사에서 확보를 해서 나름의 위기를 극복할 수 있었던 것으로 생각합니다.



## 3. 더욱 긴 오디오 입력 대한 처리

In [9]:
#%pip install pydub

In [10]:
from pydub import AudioSegment

filename = "data/채용면접_샘플_02.wav"
myaudio = AudioSegment.from_mp3(filename)

# PyDub 는 밀리초 단위로 시간을 계산합니다.
thirty_seconds = 1 * 30 * 1000  # (1초 * 30) * 1000
total_milliseconds = myaudio.duration_seconds * 1000  # 전체 길이를 밀리초로 변환

/home/kubwa/anaconda3/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [11]:
# 전체 길이를 30초로 나누어서 반복할 횟수를 계산
total_iterations = int(total_milliseconds // thirty_seconds + 1)
total_iterations

3

In [13]:
# 생성된 파일명을 저장할 리스트
output_filenames = []

for i in range(total_iterations):
    if i < total_iterations - 1:
        # 30초 단위로 오디오를 분할합니다.
        part_of_audio = myaudio[thirty_seconds * i: thirty_seconds * (i + 1)]
    else:
        # 마지막은 나머지 전체를 분할합니다.
        part_of_audio = myaudio[thirty_seconds * i:]

    output_filename = (
        # 예시: 채용면접_샘플_02-(1).mp3, 채용면접_샘플_02-(2).mp3 ...
        f"{filename[:-4]}-({i+1}).mp3"
    )

    # 분할된 오디오를 저장합니다.
    part_of_audio.export(output_filename, format="mp3")
    output_filenames.append(output_filename)

In [14]:
# 결과물(파일명) 출력
output_filenames

['data/채용면접_샘플_02-(1).mp3',
 'data/채용면접_샘플_02-(2).mp3',
 'data/채용면접_샘플_02-(3).mp3']

In [15]:
transcripts = []

for audio_filename in output_filenames:
    audio_file = open(audio_filename, "rb")  # audio file 을 읽어옵니다.

    # transcript 를 생성합니다.
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",  # 모델은 whisper-1 을 사용
        language="ko",  # 한국어를 사용
        response_format="text",  # 결과물은 text 로 출력
        temperature=0.0,
    )

    # 생성된 transcript 를 리스트에 추가합니다.
    transcripts.append(transcript)

# 전체 transcript 출력(리스트를 문자열로 변환)
final_output = "---- 분할 ---- \n".join(transcripts)
print(final_output)

뭐 제가 평소에 제가 생각했던 것 말고 다른 사람들이 주로 저한테 말했던 걸 생각해보면은 일단은 되게 한가지에 꽂히면 그거를 끝을 보는 그런 성격을 장점이라고 해줬던 것 같습니다. 항상 실행을 하는 편이었고 주변에서 저한테 너 항상 행동력이 좋다 이런 말을 들었던 것 같아요. 저도 그렇게 생각합니다.
---- 분할 ---- 
제가 하고 싶은 일이 생기고 호기심이 생기면 어쨌든 그거를 실행해봐야 다음으로 넘어갈 수 있다고 생각하고 있어요. 그래서 사실 일적인 부분 말고도 취미적인 부분에서도 되게 많은 것들을 시도했던 경험이 있고요. 단점을 말하자면 좀 주변에서 제가 많이 들었던 게 자기 비하였던 것 같아요. 제가 좀 외적으로 컴플렉스를 스스로
---- 분할 ---- 
느끼고 있는 부분들이 많은데 안 그래야지 라고 생각하면서도 스스로 좀 비하하는 말들을 많이 하게 되더라구요. 그래서 주변에서 그런 것들에 대해서 주의를 좀 많이 줬던 순간들이 있었어요. 그래서 저도 최대한 그런 것들을 좀 지양해야겠다고 스스로 좀 다 지고 있습니다.



In [16]:
example_prompt = """
당신은 채용 담당관입니다.
주어진 내용을 바탕으로, 면접자의 긍정적인 면과 부정적인 면을 나누어서 정리해 주세요.
결과물은 요약된 불렛포인트로 정리해 주세요.
한글로 작성해 주세요.
(예시)
#긍정적인면
- 
- 
-
#부정정인면
-
-  
"""


def generate_HR_opinion(temperature, prompt, transcript):
    response = client.chat.completions.create(
        model="gpt-4",
        temperature=temperature,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": transcript},
        ],
        stream=True,
    )
    final_answer = []

    # 스트림 모드에서는 completion.choices 를 반복문으로 순회
    for chunk in response:
        # chunk 를 저장
        chunk_content = chunk.choices[0].delta.content
        # chunk 가 문자열이면 final_answer 에 추가
        if isinstance(chunk_content, str):
            final_answer.append(chunk_content)
            # 토큰 단위로 실시간 답변 출력
            print(chunk_content, end="")
    return "".join(final_answer)


hr_opinion = generate_HR_opinion(0, example_prompt, final_output)

#긍정적인면
- 한 가지 일에 꽂히면 끝을 본다는 집중력
- 행동력이 좋음
- 호기심이 많고, 새로운 것을 시도하는 것을 두려워하지 않음

#부정적인면
- 자기 비하하는 경향이 있음
- 외적인 컴플렉스를 스스로 느끼고 있음
- 스스로를 지나치게 엄격하게 평가하는 경향이 있음